In [1]:
%matplotlib inline
from IPython.display import clear_output

In [22]:
import os
import pickle
import random
from collections import defaultdict
from itertools import product
import networkx as nx
from networkx.algorithms.approximation import clique
from joblib import Parallel, delayed

import Orange
import matplotlib
from matplotlib.cm import coolwarm, Spectral_r
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, label_ranking_average_precision_score
from sklearn.svm import SVR
from tqdm.notebook import tqdm

from sbm_neighbour_score import sbm_neighbour_score

In [3]:
kernels_names = [
    'Katz', 'logKatz',
    'For', 'logFor',
    'Comm', 'logComm',
    'Heat', 'logHeat',
    'NHeat', 'logNHeat',
    'SCT', 'SCCT',
    'RSP', 'FE',
    'PPR', 'logPPR',
    'ModifPPR', 'logModifPPR',
    'HeatPR', 'logHeatPR',
    'DF', 'logDF',
    'Abs', 'logAbs',
    'SP-CT'
]

shuffle = lambda x: sorted(x, key=lambda k: random.random())

def dict_argmax(dct, score_key):
    best_key = list(dct.keys())[0]
    best_val = dct[best_key]
    for k, v in dct.items():
        if v[score_key] > best_val[score_key]:
            best_key, best_val = k, v
    return best_key, best_val

CACHE_ROOT = '../../cache/cache'

def load_or_calc_and_save(filename, force_calc=False, ignore_if_exist=False):
    def decorator(func):
        def wrapped(*args, **kwargs):
            if os.path.exists(filename) and not force_calc:
                print(f'{func.__name__}: cache file {filename} found! Skip calculations')
                if not ignore_if_exist:
                    with open(filename, 'rb') as f:
                        result = pickle.load(f)
                else:
                    result = None
            else:
                print(f'{func.__name__}: RECALC {filename}.\nargs: {", ".join(args)}, kwargs: {", ".join([f"{k}={v}" for k, v in kwargs.items()])}')
                result = func(*args, **kwargs)
                with open(filename, 'wb') as f:
                    pickle.dump(result, f)
            return result
        return wrapped
    return decorator

def calc_avranks(results):  # {dataset: {classifier: accuracy}}
    ranks = defaultdict(list)
    for dataset, classifier_accuracy in results.items():
        if type(dataset) == tuple:
            dataset = '_'.join([str(x) for x in dataset])
        classifiers, accuracies = zip(*list(classifier_accuracy.items()))
        for classifier, rank in zip(classifiers, rankdata(accuracies)):
            ranks[classifier].append(rank)
    ranks = {k: np.mean(v) for k, v in sorted(ranks.items(), key=lambda x: x[0])}
    return list(ranks.values()), list(ranks.keys()), len(results)

def ytrue_to_partition(y_true):
    partition = defaultdict(list)
    for idx, class_ in enumerate(y_true):
        partition[class_].append(idx)
    return list(partition.values())

In [4]:
SBM_RESULTS_ROOT = '../../cache/kkmeans_init_sbm'
columns = [
    (100, 2, 0.05, 0.001), (100, 2, 0.05, 0.002), (100, 2, 0.05, 0.005), (100, 2, 0.05, 0.007),
    (100, 2, 0.05, 0.01), (100, 2, 0.05, 0.02), (100, 2, 0.05, 0.03), (100, 2, 0.05, 0.05),
    
    (100, 2, 0.1, 0.001), (100, 2, 0.1, 0.002), (100, 2, 0.1, 0.005), (100, 2, 0.1, 0.01),
    (100, 2, 0.1, 0.02), (100, 2, 0.1, 0.03), (100, 2, 0.1, 0.05), (100, 2, 0.1, 0.07),
    (100, 2, 0.1, 0.1),
    
    (100, 2, 0.15, 0.01), (100, 2, 0.15, 0.03), (100, 2, 0.15, 0.05), (100, 2, 0.15, 0.07),
    (100, 2, 0.15, 0.1), (100, 2, 0.15, 0.15),
    
    (100, 2, 0.2, 0.05), (100, 2, 0.2, 0.1), (100, 2, 0.2, 0.15),
    
    (100, 2, 0.3, 0.05), (100, 2, 0.3, 0.1), (100, 2, 0.3, 0.15),
    
    (102, 3, 0.1, 0.001), (102, 3, 0.1, 0.005), (102, 3, 0.1, 0.01), (102, 3, 0.1, 0.02),
    (102, 3, 0.1, 0.05), (102, 3, 0.1, 0.1),
    
    (102, 3, 0.3, 0.05), (102, 3, 0.3, 0.1), (102, 3, 0.3, 0.15),
    
    (100, 4, 0.1, 0.001), (100, 4, 0.1, 0.005), (100, 4, 0.1, 0.01), (100, 4, 0.1, 0.02),
    (100, 4, 0.1, 0.05), (100, 4, 0.1, 0.1),
    
    (100, 4, 0.3, 0.1), (100, 4, 0.3, 0.15),
    
    (150, 3, 0.1, 0.001), (150, 3, 0.1, 0.005), (150, 3, 0.1, 0.01), (150, 3, 0.1, 0.02),
    (150, 3, 0.1, 0.05), (150, 3, 0.1, 0.1),
    
    (200, 2, 0.3, 0.05), (200, 2, 0.3, 0.1), (200, 2, 0.3, 0.15),
    
    (201, 3, 0.3, 0.1),
    
    (200, 4, 0.3, 0.1), (200, 4, 0.3, 0.15)
]

def column2str(column):
    n, k, p_in, p_out = column
    return f'{n}_{k}_{p_in:.2f}_{p_out:.3f}'

datasets = [column2str(x) for x in columns]

In [5]:
with open(f'{CACHE_ROOT}/sbm_inits_bestparam_byari_individual.pkl', 'rb') as f:
    results = pickle.load(f)  # {(dataset, kernel_name, graph_idx): {scorename_initname: best_ari}}
with open(f'{CACHE_ROOT}/sbm_modularity.pkl', 'rb') as f:
    modularity_results = pickle.load(f)  # {(dataset, graph_idx): modularity}
    
for key in list(results.keys()):
    if key[0] not in datasets:
        del results[key]
        
for key in list(results.keys()):
    if key[0] not in datasets:
        del modularity_results[key]

In [6]:
results_modularity_any3 = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))  # {dataset: {graphidx: {kernel_name: best_ari}}}
for (dataset, kernel_name, graph_idx), si_ari in results.items():
    results_modularity_any3[dataset][graph_idx][kernel_name] = si_ari['modularity_any3']

In [7]:
def extract_feature(dataset_name, feature, G, partition, sp, max_clique):
    # graph-independent features
    n, k, p_in, p_out = [float(x) for x in dataset_name.split('_')]
    if feature == 'n':
        return n
    elif feature == 'k':
        return k
    elif feature == 'p_in':
        return p_in
    elif feature == 'p_out':
        return p_out
    elif feature == 'n/k':
        return n / k
    elif feature == 'p_in/p_out':
        return p_in / p_out
    
    elif feature == 'log(n)':
        return n
    elif feature == 'log(k)':
        return k
    elif feature == 'log(p_in)':
        return p_in
    elif feature == 'log(p_out)':
        return p_out
    elif feature == 'log(n/k)':
        return n / k
    elif feature == 'log(p_in/p_out)':
        return p_in / p_out
    
    elif feature == 'n/k * p_in/p_out':
        return (n / k) * (p_in / p_out)
    elif feature == 'log(n)/k * p_in/p_out':
        return np.log(n) / k * (p_in / p_out)
    elif feature == 'log(n/k) * p_in/p_out':
        return np.log(n / k) * (p_in / p_out)
    elif feature == 'log(n/k * p_in/p_out)':
        return np.log((n / k) * (p_in / p_out))
    
    elif feature == 'sbm_neighbour_score':
        return sbm_neighbour_score(int(n), int(k), p_in, p_out)
    
    # graph-dependant features
    elif feature == 'modularity':
        return nx.community.modularity(G, partition)
    elif feature == 'diameter':
        return nx.diameter(G)
    elif feature == 'density':
        return nx.density(G)
    elif feature == 'avg_deg':
        return np.mean(G.degree)
    elif feature == 'std_deg':
        return np.std(G.degree)
    elif feature == 'avg(deg | deg > avg_deg)':
        deg = np.array(G.degree)
        return np.mean(deg[deg > np.mean(deg)])
    elif feature == 'median_deg':
        return np.median(G.degree)
    elif feature == 'max_deg':
        return np.max(G.degree)
    elif feature == 'avg_sp':
        return nx.average_shortest_path_length(G)
    elif feature == 'std_sp':
        return np.std(sp)
    elif feature == 'median_sp':
        return np.median(sp)
    elif feature == 'max_sp':
        return np.max(sp)
    elif feature == 'max_clique':
        return max_clique
    elif feature == 'max_clique/(n/k)':
        return max_clique/(n/k)
    else:
        raise Exception()

# Feature importance

In [8]:
feature_names = [
    'n', 'k', 'p_in', 'p_out', 'n/k', 'p_in/p_out',
    'log(n)/k * p_in/p_out', 'n/k * p_in/p_out', 'log(n/k) * p_in/p_out', 'log(n/k * p_in/p_out)',
    'sbm_neighbour_score',
    'modularity', 'diameter', 'density', 
    'avg_deg', 'std_deg', 'avg(deg | deg > avg_deg)', 'median_deg', 'max_deg',
    'avg_sp', 'std_sp', 'median_sp', 'max_sp', 
    'max_clique', 'max_clique/(n/k)'
]

In [9]:
def prepare_column(column):
    @load_or_calc_and_save(f'{CACHE_ROOT}/feature_importance/{column}.pkl')
    def wrapper():
        X, ya, yr = [], [], []
        filename = f'{column}_100_graphs.pkl'
        with open(f'{SBM_RESULTS_ROOT}/graphs/{filename}', 'rb') as f:
            data = pickle.load(f)
        for graph_idx in range(100):
            try:
                (A, y_true), _ = data[graph_idx]
            except:
                (A, y_true) = data[graph_idx]
            G = nx.from_numpy_matrix(A)
            partition = ytrue_to_partition(y_true) 
            sp = [l for u in G for v, l in nx.single_source_shortest_path_length(G, u).items()]
            max_clique = len(clique.max_clique(G))
            features = [extract_feature(column, feature_name, G, partition, sp, max_clique) for feature_name in feature_names]
            graph_ari = [v for k, v in sorted(list(results_modularity_any3[column][graph_idx].items()), key=lambda x: x[0])]
            graph_ranks = calc_avranks({0: results_modularity_any3[column][graph_idx]})[0]

            X.append(features)
            ya.append(graph_ari)
            yr.append(graph_ranks)
        return X, ya, yr
    
    return wrapper()
    
Xy_list = Parallel(n_jobs=1)(delayed(prepare_column)(column) for column in tqdm(results_modularity_any3.keys()))

X, y, X_train, y_train, X_val, y_val = [], [], [], [], [], []
for Xi, _, yi in Xy_list:
    X.extend(Xi)
    y.extend(yi > (np.max(yi, axis=1, keepdims=True) - 0.05))
    X_train.extend(Xi[:70])
    y_train.extend(yi[:70] > (np.max(yi[:70], axis=1, keepdims=True) - 0.05))
    X_val.extend(Xi[70:])
    y_val.extend(yi[70:] > (np.max(yi[70:], axis=1, keepdims=True) - 0.05))
    
X, y, X_train, y_train, X_val, y_val = np.array(X), np.array(y), np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val)

wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.001.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.002.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.005.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.007.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.010.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.020.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.030.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.05_0.050.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.10_0.001.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/100_2_0.10_0.002

In [10]:
X_train[0]

array([1.00000000e+02, 2.00000000e+00, 5.00000000e-02, 1.00000000e-03,
       5.00000000e+01, 5.00000000e+01, 1.15129255e+02, 2.50000000e+03,
       1.95601150e+02, 7.82404601e+00, 9.08495868e-01, 4.84735468e-01,
       1.60000000e+01, 2.76767677e-02, 2.61200000e+01, 3.10514992e+01,
       6.30000000e+01, 5.00000000e+00, 9.90000000e+01, 6.11515152e+00,
       2.82833591e+00, 6.00000000e+00, 1.60000000e+01, 3.00000000e+00,
       6.00000000e-02])

In [11]:
estimator = RandomForestClassifier(n_estimators=100)
estimator.fit(X_train, y_train)
pd.DataFrame([{'feature': k, 'importance': v} for k, v in sorted(zip(feature_names, estimator.feature_importances_), key=lambda x: -x[1])])

,feature,importance
0,sbm_neighbour_score,0.128849
1,modularity,0.102120
2,std_sp,0.082625
3,avg_sp,0.081584
4,density,0.078435
5,std_deg,0.078301
6,log(n/k * p_in/p_out),0.064817
7,avg_deg,0.063471
8,n/k * p_in/p_out,0.060190
9,log(n)/k * p_in/p_out,0.049334


In [69]:
estimator = RandomForestClassifier(n_estimators=100)
selector = RFE(estimator, n_features_to_select=1, verbose=1)
selector = selector.fit(X_train, y_train)
print(list(zip(feature_names, selector.support_)))
print(list(zip(feature_names, selector.ranking_)))

Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
[('n', False), ('k', False), ('p_in', False), ('p_out', False), ('n/k', False), ('p_in/p_out', False), ('log(n)/k * p_in/p_out', False), ('n/k *

In [70]:
pd.DataFrame(zip(feature_names, selector.support_, selector.ranking_), columns=['feature', 'to choose', 'rank']).sort_values('rank')

,feature,to choose,rank
11,modularity,True,1
20,std_sp,False,2
10,sbm_neighbour_score,False,3
15,std_deg,False,4
9,log(n/k * p_in/p_out),False,5
19,avg_sp,False,6
13,density,False,7
7,n/k * p_in/p_out,False,8
14,avg_deg,False,9
6,log(n)/k * p_in/p_out,False,10


In [81]:
feature_names[10]

'sbm_neighbour_score'

In [98]:
estimator = RandomForestClassifier(n_estimators=100)
estimator.fit(X_train[:, [10, 19, 20]], y_train)

RandomForestClassifier()

In [99]:
print('\n'.join([f'{k}\t{v:.3f}' for k, v in sorted(zip(np.array(feature_names)[[10, 19, 20]], estimator.feature_importances_), key=lambda x: -x[1])]))

sbm_neighbour_score	0.373
std_sp	0.320
avg_sp	0.308


In [100]:
y_pred = estimator.predict(X_val[:, [10, 19, 20]])

In [101]:
accuracy_score(y_val.ravel(), y_pred.ravel())

0.8367380952380953

In [102]:
f1_score(y_val.ravel(), y_pred.ravel())

0.7067527691057607